In [2]:
import jax
import numpy as np
import jax.numpy as jnp
import pylab as plt
import scipy
import pandas as pd

## Data Analysis

In [8]:
df = pd.read_csv('songsDataset.csv')
df.columns = ['userID', 'songID', 'rating']
df.head()

,userID,songID,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5


In [12]:
print(f'Number of unique users: {len(set(df["userID"].tolist()))}')
print(f'Number of unique songs: {len(set(df["songID"].tolist()))}')
print(f'All ratings = {set(df["rating"].tolist())}')  # no invalid ratings

Number of unique users: 200000
Number of unique songs: 127771
All ratings = {1, 2, 3, 4, 5}


In [22]:
print(f'Dataset before deduplication: {len(df)}')
df = df.drop_duplicates(subset=['userID', 'songID']).reset_index(drop=True)
print(f'Dataset after deduplication: {len(df)}')  # no (same user, song, diff rating)

Dataset before deduplication: 2000000
Dataset after deduplication: 2000000


In [19]:
user_rating_counts = df.groupby('userID')['songID'].count().reset_index()
user_rating_counts.columns = ['userID', 'song_count']
print(f'A user gives ratings to {set(user_rating_counts["song_count"])} songs')
user_rating_counts

A user gives ratings to {10} songs


,userID,song_count
0,0,10
1,1,10
2,2,10
3,3,10
4,4,10
...,...,...
199995,199995,10
199996,199996,10
199997,199997,10
199998,199998,10


In [21]:
song_rating_counts = df.groupby('songID')['userID'].count().reset_index()
song_rating_counts.columns = ['songID', 'user_count']
print(f'A song is rated by {set(song_rating_counts["user_count"])} users')
song_rating_counts

A song is rated by {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

,songID,user_count
0,0,5
1,1,2
2,2,4
3,3,18
4,4,2
...,...,...
127766,136731,6
127767,136732,7
127768,136733,2
127769,136734,23


## Matrix Fatorization

In [ ]:
# code from lecture
@jax.jit
def loss(U_single, F_single, rating_single):
    '''compute the error for a single pair of user and film'''
    pred = jnp.dot(U_single, F_single)
    error = rating_single - pred
    return jnp.mean(error ** 2)
loss_grad = jax.jit( jax.grad(loss, argnums=(0, 1)) )


def mse(U_batch, F_batch, ratings_batch):
    '''compute the MSE on a batch of users * films'''
    predictions = np.sum(U_batch * F_batch, axis=1)
    errors = ratings_batch - predictions
    return np.mean(errors ** 2)


# need to generate the U, F, Ratings, Obs_users, Obs_films from df to run this :) 
# refactor the arguments later
niter = 100
learning_rate = 0.05
def naive_matrix_factorization(U, F, Ratings, Obs_users, Obs_films, niter, learning_rate):
    for it in range(niter):
        for index in range(1000):
            # fetch a random user, film and rating
            index = np.random.randint(Ratings.shape[0])  # check here
            user_id = Obs_users[index]
            film_id = Obs_films[index]
            rating = Ratings[index]
            # lt's compute the gradient
            U_grad, F_grad = loss_grad(U[user_id], F[film_id], rating)
            # lt's do the update
            U[user_id] -= learning_rate * U_grad
            F[film_id] -= learning_rate * F_grad
        if it % 10 == 0:
            U_batch = U[Obs_users[:1000]]
            F_batch = F[Obs_films[:1000]]
            ratings_batch = Ratings[:1000]
            mse_current = mse(U_batch, F_batch, ratings_batch)
            print(f'iter {it}, mse = {mse_current:.2f}')